In [ ]:
# Import the zipfile module
import zipfile

# Import the drive module from google.colab
from google.colab import drive

# Mount your Google Drive to the Colab VM
drive.mount('/content/drive')

# Specify the zip file name and the path to it
zip_file_name = "v3.zip" # Change this to your file name
zip_file_path = "/content/drive/MyDrive/" # Change this to your file path

# Create a ZipFile object and open the zip file
zip_file = zipfile.ZipFile(zip_file_path + zip_file_name, 'r')

# Extract all the files from the zip file to the current working directory
zip_file.extractall()

# Close the zip file
zip_file.close()


Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import pandas as pd

# Load the data
with open('/content/v2/en-hi/train.hi', 'r', encoding='utf-8') as f:
    hi_lines = f.readlines()

with open('/content/v2/en-hi/train.en', 'r', encoding='utf-8') as f:
    en_lines = f.readlines()

# Check if both files have the same number of lines
assert len(hi_lines) == len(en_lines), "Files have different number of lines"

# Create a DataFrame
df = pd.DataFrame({
    'Hindi': hi_lines,
    'English': en_lines
})


df['Hindi'] = df['Hindi'].str.strip()
df['English'] = df['English'].str.strip()





In [ ]:
df

,Hindi,English
0,आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाल...,"However, Paes, who was partnering Australia's ..."
1,और जो शख्स (अपने आमाल का) बदला दुनिया ही में च...,"Whosoever desires the reward of the world, wit..."
2,"जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि ...",The value of insects in the biosphere is enorm...
3,आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को,Mithali To Anchor Indian Team Against Australi...
4,"8 सितम्‍बर, 2016 को माननीय राष्‍ट्रपति की स्‍व...",After the assent of the Honble President on 8t...
...,...,...
10125701,"अक्षय कुमार, करीना कपूर और प्रियंका चोपड़ा ने ...","Produced by Subhash Ghai, it stars Akshay Kuma..."
10125702,बड़ा सवाल यह है कि अगर ऐसी श्रेणी के कर्मचारियो...,"The bigger question, though, is this: if such ..."
10125703,और संभवतः और भी प्रभावशाली है कि यदि लोग हमारे...,And perhaps even more impressive is that if pe...
10125704,5 मिनट में मूवी डाउनलोड,1 Movie download in 5 minutes


In [ ]:
!pip install sentencepiece

In [ ]:
import torch
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd


# Specify the model name
model_name = 'Helsinki-NLP/opus-mt-hi-en'

# Load the preprocessed data
df=df.sample(frac=0.25)
# Load the tokenizer
tokenizer = MarianTokenizer.from_pretrained(model_name)

# Tokenize the input and output texts
input_texts = tokenizer.prepare_seq2seq_batch(df['Hindi'], return_tensors='pt', padding=True, truncation=True, max_length=512)
output_texts = tokenizer.prepare_seq2seq_batch(df['English'], return_tensors='pt', padding=True, truncation=True, max_length=512)

# Load the model
model = MarianMTModel.from_pretrained(model_name)

# Specify the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
input_texts.to(device)
output_texts.to(device)

# Training loop
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(4):  # Number of epochs
    optimizer.zero_grad()
    outputs = model(**input_texts, labels=output_texts['input_ids'])
    loss = outputs.loss
    loss.backward()
    optimizer.step()

    print(f'Epoch: {epoch}, Loss: {loss.item()}')

# Save the model
model.save_pretrained('./trained_model')
model.push_to_hub(repository_name='nlp-q3')
